# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [110]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [111]:
year = "2022"

In [112]:
input_data='./input_data/'
output_data='./output_data/'

if year != '':
    output_data += year + '/'

input_data, output_data

('./input_data/', './output_data/2022/')

Set the model name below. This is used to lookup model parameters from the models list file.

- Base Case: Base Case
- MW162: Capacity reduced to 162MW
- Headpond50cm: Headpond level increased by 50cm
- EWRAllCs: All river reaches at a C rather than B environmental assurance level
- Extend Head: Minimum head reduce from 10m to 9m
- Tailwater Rise 50cm: Tailwater curve raised by 50cm
- Tailwater Fall 50cm: Tailwater curve dropped by 50cm

In [113]:
model_name='MW270'
dryrun=False #Dont write output files
output_separate_years=False

In [114]:
if year != '':
    model_file=input_data + year + "/models.csv"
else:
    model_file=input_data + "models.csv"

models=pd.read_csv(model_file).set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000

model

OutputPrefix                                      mw270
Description                 Capacity increased to 270MW
EWRCatSet                              Recommendation 1
TailwaterLift                                       0.0
HeadpondLift                                        0.0
Capacity                                            180
MinimumHead                                          10
MinimumFlowUnit                                      50
FixedTailwater                                        0
UnitsAvailable                                        4
RatedDischarge                                      250
RatedHead                                          21.7
RatedTurbineEfficiency                            0.943
RatedGeneratorEfficiency                           0.98
MaximumLoadFactor                                   1.1
RatedPower                                          NaN
Mean                                                NaN
P95                                             

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [115]:
daily_file = output_data + out_prefix + '_pe_daily.csv'
daily=pd.read_csv(daily_file) #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily_file, daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35794 entries, 1924-10-01 to 2022-09-30
Data columns (total 83 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LaggedDate                  35794 non-null  object 
 1   VicFalls                    35794 non-null  float64
 2   Conversion                  35794 non-null  float64
 3   Flow                        35794 non-null  float64
 4   Exceedance                  35794 non-null  float64
 5   Year                        35794 non-null  int64  
 6   Month                       35794 non-null  int64  
 7   Day                         35794 non-null  int64  
 8   MonthId                     35794 non-null  float64
 9   WaterYear                   35794 non-null  int64  
 10  WaterMonth                  35794 non-null  int64  
 11  WaterDay                    35794 non-null  int64  
 12  WaterWeek                   35794 non-null  int64  
 13  Volume        

('./output_data/2022//scenarios/mw270_pe_daily.csv', None)

In [116]:
monthly=daily.groupby("MonthId").mean(numeric_only=True).drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
   FlowAvailableForGeneration_min=('FlowAvailableForGeneration','min')
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,Flow_difference_pct,EWRChannelA,EWRChannelC,...,PotentialPower3u,PotentialPower4u,Power_max,Power_min,Turbines_max,Turbines_min,Energy,FlowAvailableForGeneration_min,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
1924.10,136.771526,0.998452,1924.0,10.0,1924.0,1.0,0.610130,0.020579,28.193548,4.322581,...,170.816235,224.367670,18.826188,0.000000,1,0,8768.362487,41.409487,31,0.065475
1924.11,153.655969,0.992400,1924.0,11.0,1924.0,2.0,2.353660,0.014794,23.000000,4.000000,...,170.891048,224.480183,31.164738,17.169920,1,1,15905.545468,82.587158,30,0.122728
1924.12,276.157068,0.831032,1924.0,12.0,1924.0,3.0,9.160736,0.029641,26.000000,4.000000,...,169.568195,222.507726,81.083810,29.451895,2,1,35556.609210,134.122845,31,0.265506
1925.01,704.059970,0.459290,1925.0,1.0,1924.0,4.0,44.430930,0.055882,37.129032,5.677419,...,160.150750,208.864290,192.833273,72.895921,4,2,88632.131735,345.554562,31,0.661829
1925.02,1210.241015,0.279000,1925.0,2.0,1924.0,5.0,25.090361,0.020239,49.000000,8.000000,...,148.809163,192.819255,192.063521,177.562886,4,4,123977.193101,964.382002,28,1.024944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.05,2785.549155,0.106677,2022.0,5.0,2021.0,8.0,29.266761,0.011143,61.774194,13.677419,...,114.659863,145.981415,160.470564,135.903077,4,4,107923.727108,1898.594927,31,0.805882
2022.06,1534.964803,0.236667,2022.0,6.0,2021.0,9.0,40.784479,0.027095,62.000000,12.000000,...,141.335707,182.484865,188.065526,141.027559,4,3,122858.047763,739.060566,30,0.947979
2022.07,668.630149,0.469161,2022.0,7.0,2021.0,10.0,16.433611,0.023411,58.000000,11.000000,...,160.960224,209.996393,142.555242,55.267895,3,1,66229.397936,259.618631,31,0.494544


In [117]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [118]:
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2017,4855.789688,171.188037,0.555972,0.055607,25.389345,10.817003,60.375384,977.246095,964.400523,189.826820,20.659253,843190.565090,0.534748
2018,1103.622889,158.591340,0.523314,0.109287,25.434442,20.412637,60.261322,968.225672,964.400523,180.525026,18.007516,725034.935667,0.459814
2019,4709.551434,130.594892,0.447791,0.054678,25.380054,11.084619,60.261322,976.978479,964.400523,191.472684,16.155691,808413.901865,0.512693
2020,3732.969062,261.920014,0.473687,0.050978,24.964581,12.948949,58.743688,975.114149,964.747680,192.221775,33.750483,914589.141981,0.580029
2021,3116.982122,265.901164,0.534663,0.069840,24.969474,14.266934,58.044919,973.796164,964.763605,192.073745,32.330286,892669.152619,0.566127


In [119]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))

In [120]:
yearly['PropDays1uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterYear').count()
yearly['PropDays2uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterYear').count()
yearly['PropDays3uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterYear').count()
yearly['PropDays4uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterYear').count()

In [121]:
daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

WaterYear
1924.0     58
1925.0     67
1926.0     54
1927.0     68
1928.0    107
         ... 
2016.0     54
2017.0     40
2019.0     63
2020.0     37
2021.0     68
Name: Date, Length: 88, dtype: int64

In [122]:
yearly['FstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)


## Power and energy exceedance

In [123]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed

,Power
Exceedance,
0.00,194.688781
0.05,183.485604
0.10,174.526370
0.15,164.380763
0.20,155.742263
0.25,147.317384
0.30,138.276012
0.35,128.284364
0.40,114.637541


In [124]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc

,Energy
0.00,1.028049e+06
0.05,9.657507e+05
0.10,9.308606e+05
0.15,9.111523e+05
0.20,8.955850e+05
0.25,8.810371e+05
0.30,8.679354e+05
0.35,8.583803e+05
0.40,8.532784e+05
0.45,8.496906e+05


In [125]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [126]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [127]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,862026.299744,0.546693
1926,784319.550379,0.497412
1927,860446.014394,0.545691
1928,837078.076369,0.530871
1929,762793.408552,0.483760


## Monthly Day
15th of every month

In [128]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [129]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-19,2022,9,19,2021,286,0.42,3,40,6,4,...,0.92,0.0,0.0,0.0,36.7,0.0,0.0,0.0,36.7,881
2022-09-20,2022,9,20,2021,283,0.42,3,40,6,4,...,0.92,0.0,0.0,0.0,36.1,0.0,0.0,0.0,36.1,866
2022-09-21,2022,9,21,2021,281,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.7,0.0,0.0,0.0,35.7,855
2022-09-22,2022,9,22,2021,282,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.7,0.0,0.0,0.0,35.7,856
2022-09-23,2022,9,23,2021,279,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.2,0.0,0.0,0.0,35.2,844
2022-09-24,2022,9,24,2021,274,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,34.2,0.0,0.0,0.0,34.2,820
2022-09-25,2022,9,25,2021,272,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,33.8,0.0,0.0,0.0,33.8,811
2022-09-26,2022,9,26,2021,273,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,33.8,0.0,0.0,0.0,33.8,812
2022-09-27,2022,9,27,2021,268,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,32.9,0.0,0.0,0.0,32.9,788


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [130]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean(numeric_only=True).drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2021      42         604.065639    0.490143        13.445609   
          43         534.657921    0.535143         8.809745   
          44         483.056208    0.571429         5.767399   
          45         442.671726    0.608429         6.118159   
          46         410.196894    0.642143         3.636412   
          47         380.962525    0.678286         4.358470   
          48         352.031809    0.710714         3.807089   
          49         329.501263    0.742429         2.972353   
          50         309.458675    0.770286         2.746306   
          51         288.344242    0.803714         2.922429   
          52         271.680619    0.832000         2.467244   
          53         266.795709    0.842000         0.000000   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2021      42                    0.022107    58.000000    11.000000   
          43                    0.016522    58.000000    11.000000   
          44                    0.011917    55.714286     9.857143   
          45                    0.013841    54.000000     9.000000   
          46                    0.008876    54.000000     9.000000   
          47                    0.011451    54.000000     9.000000   
          48                    0.010790    52.000000     8.571429   
          49                    0.009019    40.000000     6.000000   
          50                    0.008860    40.000000     6.000000   
          51                    0.010093    40.000000     6.000000   
          52                    0.009078    40.000000     6.000000   
          53                    0.000000    40.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2021      42           11.000000    11.000000    136.000000  227.000000  ...   
          43           11.000000    11.000000    136.000000  227.000000  ...   
          44            9.285714     8.142857    119.428571  202.428571  ...   
          45            8.000000     6.000000    107.000000  184.000000  ...   
          46            8.000000     6.000000    107.000000  184.000000  ...   
          47            8.000000     6.000000    107.000000  184.000000  ...   
          48            7.428571     5.571429    101.000000  174.571429  ...   
          49            4.000000     3.000000     65.000000  118.000000  ...   
          50            4.000000     3.000000     65.000000  118.000000  ...   
          51            4.000000     3.000000     65.000000  118.000000  ...   
          52            4.000000     3.000000     65.000000  118.000000  ...   
          53            4.000000     3.000000     65.000000  118.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2021      42                23.408955         22.928655          966.494083   
          43                23.787421         23.472421          966.116828   
          44                23.968052         23.461976          965.859956   
          45                24.015706         23.675932          965.667913   
          46                24.126441         23.942470          965.496582   
          47                24.379998         24.173100          965.368495   
          48                24.573070         24.387052          965.227217   
          49                24.565017         24.410736          965.096385   
          50                24.722452         24.594515          964.988381   
          51                24.863838         24.739715          964.899202   
          52                24.969474         24.878337          964.818539   
 

In [131]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [132]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Power_P95']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.05)
calmonthly['Power_P90']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.10)

calmonthly['FlowAvailableForGeneration_P95']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.05)
calmonthly['FlowAvailableForGeneration_P50']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.5)

calmonthly['PotentialPower1u_P95']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower1u_P50']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower2u_P95']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower2u_P50']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower3u_P95']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower3u_P50']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower4u_P95']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower4u_P50']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.5)


calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower1u_P95,PotentialPower1u_P50,PotentialPower2u_P95,PotentialPower2u_P50,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,56.819208,57.492010,113.117678,114.521046,168.274749,170.524376,220.586512,223.930391,6718.133574,0.284415
2.0,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,56.354852,57.273064,112.154248,114.063598,166.742788,169.789243,218.337473,222.833534,8817.972477,0.302274
3.0,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,54.364706,56.406706,108.055427,112.262591,160.298746,166.916908,209.044677,218.597291,14800.739425,0.283147
4.0,Jan,1,0.222306,0.174848,0.178932,0.097430,150.0,117.801514,120.000000,73.0,...,50.052735,54.789506,99.270604,108.926130,146.722517,161.657360,190.009420,210.980198,20599.384042,0.242451
5.0,Feb,2,0.186837,0.127288,0.136113,0.023905,197.0,138.857520,137.000000,85.0,...,37.258886,52.177329,73.434004,103.581342,107.361339,153.340494,136.160168,199.185162,18474.187752,0.186006
6.0,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,24.045337,42.516127,40.706561,84.008618,45.878586,123.365621,29.302762,157.801001,30856.737470,0.297509
7.0,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,26.246990,38.541728,51.375042,76.006792,74.206019,111.236019,70.351384,141.352854,28769.128003,0.296261
8.0,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,33.569146,43.077127,66.035616,85.141005,96.223175,125.089463,120.237097,160.156264,17505.568637,0.153502
9.0,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,44.442135,50.800486,87.897457,100.780332,129.288561,149.021773,165.900499,193.153368,23189.374800,0.217358


In [133]:
calmonthly[['Power_P90','Power_P95']]

,Power_P90,Power_P95
WaterMonth,,
1.0,22.546154,20.273702
2.0,26.453554,23.744343
3.0,44.118095,39.923223
4.0,76.282734,69.200428
5.0,108.440060,96.377584
6.0,94.784074,0.000000
7.0,94.784074,0.000000
8.0,120.511456,107.304874
9.0,82.395772,64.746353


In [134]:
calmonthly['PropYearsMinAvailFlowGt1UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt2UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt3UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGtCanalRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>canal_capacity).groupby('WaterMonth').count()/len(yearly.index)
calmonthly.fillna(0)
calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,168.274749,170.524376,220.586512,223.930391,6718.133574,0.284415,0.030612,NaN,NaN,NaN
2.0,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,166.742788,169.789243,218.337473,222.833534,8817.972477,0.302274,0.051020,NaN,NaN,NaN
3.0,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,160.298746,166.916908,209.044677,218.597291,14800.739425,0.283147,0.265306,0.020408,NaN,NaN
4.0,Jan,1,0.222306,0.174848,0.178932,0.097430,150.0,117.801514,120.000000,73.0,...,146.722517,161.657360,190.009420,210.980198,20599.384042,0.242451,0.969388,0.224490,0.051020,NaN
5.0,Feb,2,0.186837,0.127288,0.136113,0.023905,197.0,138.857520,137.000000,85.0,...,107.361339,153.340494,136.160168,199.185162,18474.187752,0.186006,1.000000,0.857143,0.377551,0.122449
6.0,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,45.878586,123.365621,29.302762,157.801001,30856.737470,0.297509,1.000000,0.979592,0.806122,0.551020
7.0,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,74.206019,111.236019,70.351384,141.352854,28769.128003,0.296261,1.000000,1.000000,0.959184,0.816327
8.0,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,96.223175,125.089463,120.237097,160.156264,17505.568637,0.153502,1.000000,0.948980,0.897959,0.734694
9.0,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,129.288561,149.021773,165.900499,193.153368,23189.374800,0.217358,0.948980,0.602041,0.275510,0.061224


In [135]:
calmonthly['PropYears1uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears2uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears3uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears4uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterMonth').count()/len(yearly.index)

calmonthly.fillna(0)
calmonthly


,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow,PropYears1uPotentialGtRatedPower,PropYears2uPotentialGtRatedPower,PropYears3uPotentialGtRatedPower,PropYears4uPotentialGtRatedPower
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1.0,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,6718.133574,0.284415,0.030612,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
2.0,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,8817.972477,0.302274,0.051020,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
3.0,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,14800.739425,0.283147,0.265306,0.020408,NaN,NaN,1.000000,1.000000,1.000000,1.000000
4.0,Jan,1,0.222306,0.174848,0.178932,0.097430,150.0,117.801514,120.000000,73.0,...,20599.384042,0.242451,0.969388,0.224490,0.051020,NaN,0.959184,0.948980,0.938776,0.908163
5.0,Feb,2,0.186837,0.127288,0.136113,0.023905,197.0,138.857520,137.000000,85.0,...,18474.187752,0.186006,1.000000,0.857143,0.377551,0.122449,0.744898,0.724490,0.693878,0.632653
6.0,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,30856.737470,0.297509,1.000000,0.979592,0.806122,0.551020,0.306122,0.295918,0.285714,0.204082
7.0,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,28769.128003,0.296261,1.000000,1.000000,0.959184,0.816327,0.153061,0.132653,0.112245,0.071429
8.0,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,17505.568637,0.153502,1.000000,0.948980,0.897959,0.734694,0.193878,0.163265,0.142857,0.122449
9.0,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,23189.374800,0.217358,0.948980,0.602041,0.275510,0.061224,0.673469,0.632653,0.561224,0.448980


## Flow duration curve

In [136]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean(numeric_only=True).drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8809.879593,0.001431,0.761174,141.407689,0.016459,0.026667,1.097222,66.083333,12.430556,14.861111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6389.293248,0.004989,0.552035,109.134863,0.016749,0.043611,1.100000,72.427778,13.838889,17.688889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.583995,15.026123,10.259092,0.000000
0.010,5563.531659,0.009983,0.480689,65.252467,0.011744,0.083204,1.320442,72.668508,13.574586,17.591160,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.278510,47.444249,68.329705,19.568042
0.015,5203.203065,0.015028,0.449557,89.050316,0.017197,0.101611,1.388889,71.366667,12.933333,16.600000,...,22.301253,22.301253,22.301253,89.205012,2140.920287,0.017160,25.808464,50.497232,72.888267,90.135119
0.020,4839.769106,0.020054,0.418156,93.641639,0.019328,0.110598,1.402174,69.554348,13.195652,16.597826,...,23.983459,23.983459,23.983459,95.933835,2302.412046,0.019839,27.516389,53.908391,77.989452,96.882185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,181.295707,0.980401,0.015664,2.697233,0.014865,0.893882,4.447368,28.546053,4.355263,2.782895,...,NaN,NaN,NaN,24.774301,594.583236,0.136628,57.650379,114.852798,171.059598,224.733672
0.985,172.231000,0.985087,0.014881,1.883828,0.010890,0.910510,4.525510,27.591837,4.229592,2.571429,...,NaN,NaN,NaN,23.383749,561.209966,0.135764,57.629686,114.809394,170.989442,224.628081
0.990,163.652909,0.990268,0.014140,1.881114,0.011482,0.944524,4.857143,25.636905,4.333333,2.392857,...,NaN,NaN,NaN,22.702256,544.854148,0.138660,57.610013,114.768144,170.922794,224.527834


## Add overall summary data to the model file

In [137]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.457184,0.289998,0.245072,0.223429,0.134203,0.121133,0.093593,0.156767,0.097678,0.071936,-0.053329,-0.10399,-0.037934,-0.161246,-0.169127,-0.222255,-0.243456,-0.267082,-0.20208
1,"[1.0, 1.0]","[0.2591979694555016, 0.6551704906982667]","[0.05423304546010985, 0.5257621449557776]","[-0.004282999775156449, 0.49442738215719606]","[-0.03519588242867447, 0.4820528852666218]","[-0.13187959856218498, 0.4002864628773537]","[-0.147590080149018, 0.3898563091517505]","[-0.17726188795809447, 0.36444834606175636]","[-0.11535252455227216, 0.42888723535452766]","[-0.1779596778900561, 0.3733149068635786]","[-0.20505436724981363, 0.3489272111550549]","[-0.3310507448017055, 0.22439354777619783]","[-0.38211297311055137, 0.17413354488602928]","[-0.31757753275433087, 0.24170887942851227]","[-0.44109121815189745, 0.1185984715766745]","[-0.4525902174617752, 0.11433656118869909]","[-0.5096461872614093, 0.06513709426022413]","[-0.5375075844091167, 0.05059629824264056]","[-0.5689314233869684, 0.03476781430675674]","[-0.5130546939650239, 0.10889536767760724]"


In [138]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,7,1934-03-24,1934-03-30,1934.03,182.982004
1947,42,1948-03-03,1948-04-13,1948.03,103.672655
1951,22,1952-03-01,1952-03-22,1952.03,102.773576
1952,13,1953-03-21,1953-04-02,1953.03,189.187831
1955,17,1956-05-03,1956-05-19,1956.05,113.435749
1956,27,1957-03-02,1957-03-28,1957.03,103.307792
1957,61,1958-02-03,1958-04-04,1958.02,107.425808
1960,24,1961-04-01,1961-04-24,1961.04,96.338819
1961,17,1962-04-02,1962-04-18,1962.04,100.872574


In [139]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,18.826188


In [140]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.at[model_name,'RatedPower']=rated_power_unit

models.at[model_name,'P95FirmPower']=power_exceed.iloc[19]
models.at[model_name,'P90FirmPower']=power_exceed.iloc[18]

models.loc[model_name]



C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_25656\3956752596.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  models.at[model_name,'P95FirmPower']=power_exceed.iloc[19]
C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_25656\3956752596.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  models.at[model_name,'P90FirmPower']=power_exceed.iloc[18]


OutputPrefix                                      mw270
Description                 Capacity increased to 270MW
EWRCatSet                              Recommendation 1
TailwaterLift                                       0.0
HeadpondLift                                        0.0
Capacity                                            180
MinimumHead                                          10
MinimumFlowUnit                                      50
FixedTailwater                                        0
UnitsAvailable                                        4
RatedDischarge                                      250
RatedHead                                          21.7
RatedTurbineEfficiency                            0.943
RatedGeneratorEfficiency                           0.98
MaximumLoadFactor                                   1.1
RatedPower                                    49.182038
Mean                                       837946.41559
P95                                       721588

## Save Files

In [141]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

    if year != '':
        models.to_csv(input_data + year + "/models.csv")
    else:
        models.to_csv(input_data + "models.csv")

In [142]:
if output_separate_years:
    years=daily['WaterYear'].unique().tolist()
    for year in years:
        days=daily.loc[daily.WaterYear==year]
        days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 